# Task 1
爬取豆瓣 Top250 or 最新电影(例如战狼 2、敦刻尔克、蜘蛛侠、银魂)的短评数据，保证抓取尽量完整


In [1]:
import requests 
import time
from lxml import etree
import sys
from PIL import Image 
import os
from io import BytesIO

In [2]:
headers = {'User-Agent':'Mozilla/5.0'}

In [3]:
session = requests.session() #Provides cookie persistence, connection-pooling, and configuration.
login_r = session.get('https://www.douban.com/login', headers=headers)
#检查登陆状态
login_r.status_code

200

In [4]:
#check编码
login_r.encoding

'utf-8'

In [5]:
#define对象
s = etree.HTML(login_r.content) 

In [6]:
post_data = {'source':'None', 
             'redir':'https://www.douban.com', 
             'form_email':'1156906182@qq.com', 
             'form_password':'WCXJ8679!', 
             'login':'登录'}
# get verification image url
captcha_img_url=s.xpath('//img[@id="captcha_image"]/@src')
import matplotlib.pyplot as plt
if captcha_img_url:
    pic_request = requests.get(captcha_img_url[0])
    img = Image.open(BytesIO(pic_request.content))  
    print('请输入你看到的字母:')
    plt.figure('code')
    plt.imshow(img)
    plt.show()
    string = input('请输入验证码：')
    # save image letter into captcha-solution
    post_data['captcha-solution'] = string
    captcha_id = s.xpath('//input[@name="captcha-id"]/@value')
    post_data['captcha-id'] = captcha_id[0]

# login using finalized post info
r = session.post('https://accounts.douban.com/login', data = post_data)

In [7]:
url = "https://movie.douban.com"
#用session访问
test = session.get(url, headers=headers)
test.status_code #检查访问状态

200

In [8]:
#拼出多页url
html = ' '
for i in range(10):
    url = 'https://movie.douban.com/top250?start='+ str(i*25) + '&filter='
    print(url)
    page = session.get(url, headers=headers) #用session登陆状态访问
    html += page.text
time.sleep(2)

https://movie.douban.com/top250?start=0&filter=
https://movie.douban.com/top250?start=25&filter=
https://movie.douban.com/top250?start=50&filter=
https://movie.douban.com/top250?start=75&filter=
https://movie.douban.com/top250?start=100&filter=
https://movie.douban.com/top250?start=125&filter=
https://movie.douban.com/top250?start=150&filter=
https://movie.douban.com/top250?start=175&filter=
https://movie.douban.com/top250?start=200&filter=
https://movie.douban.com/top250?start=225&filter=


In [9]:
print(html[:1000])

 <!DOCTYPE html>
<html lang="zh-cmn-Hans" class="">
<head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <meta name="renderer" content="webkit">
    <meta name="referrer" content="always">
    <title>
豆瓣电影 Top 250
</title>
    
    <meta name="baidu-site-verification" content="cZdR4xxR7RxmM4zE" />
    <meta http-equiv="Pragma" content="no-cache">
    <meta http-equiv="Expires" content="Sun, 6 Mar 2005 01:00:00 GMT">
    
    <link rel="apple-touch-icon" href="https://img3.doubanio.com/f/movie/d59b2715fdea4968a450ee5f6c95c7d7a2030065/pics/movie/apple-touch-icon.png">
    <link href="https://img3.doubanio.com/f/shire/420c6a4b676c73bc6af48dfcdd18b662f5c223d7/css/douban.css" rel="stylesheet" type="text/css">
    <link href="https://img3.doubanio.com/f/shire/ae3f5a3e3085968370b1fc63afcecb22d3284848/css/separation/_all.css" rel="stylesheet" type="text/css">
    <link href="https://img3.doubanio.com/f/movie/8864d3756094f5272d3c93e30ee2e324665855b0/css/movie/base/

In [10]:
s = etree.HTML(html)
movies = s.xpath('//*[@class="hd"]')

In [11]:
len(movies)

250

In [12]:
titles = []
movie_urls = []

for movie in movies:
    title = movie.xpath('.//a/span[1]/text()')[0]
    url = movie.xpath('.//a/@href')[0]
    #print(title)
    #print(url)
    titles.append(title)
    movie_urls.append(url)

In [13]:
import pandas as pd
df = pd.DataFrame(list(zip(titles, movie_urls)))
df.head()

,0,1
0,肖申克的救赎,https://movie.douban.com/subject/1292052/
1,霸王别姬,https://movie.douban.com/subject/1291546/
2,这个杀手不太冷,https://movie.douban.com/subject/1295644/
3,阿甘正传,https://movie.douban.com/subject/1292720/
4,美丽人生,https://movie.douban.com/subject/1292063/


In [14]:
# 目标：每个电影有一个comment page url list，共250个list构成cmt_urls
cmt_urls = []
for url in movie_urls:
    links = []
    # 每部影片抓取10页短评
    for i in range(10):
        link = url + 'comments?start=' + str(i*20) + '&limit=20&sort=new_score&status=P&percent_type='
        links.append(link)
    cmt_urls.append(links) 
    

In [15]:
df['links'] = cmt_urls
df.loc[0, 'links']

['https://movie.douban.com/subject/1292052/comments?start=0&limit=20&sort=new_score&status=P&percent_type=',
 'https://movie.douban.com/subject/1292052/comments?start=20&limit=20&sort=new_score&status=P&percent_type=',
 'https://movie.douban.com/subject/1292052/comments?start=40&limit=20&sort=new_score&status=P&percent_type=',
 'https://movie.douban.com/subject/1292052/comments?start=60&limit=20&sort=new_score&status=P&percent_type=',
 'https://movie.douban.com/subject/1292052/comments?start=80&limit=20&sort=new_score&status=P&percent_type=',
 'https://movie.douban.com/subject/1292052/comments?start=100&limit=20&sort=new_score&status=P&percent_type=',
 'https://movie.douban.com/subject/1292052/comments?start=120&limit=20&sort=new_score&status=P&percent_type=',
 'https://movie.douban.com/subject/1292052/comments?start=140&limit=20&sort=new_score&status=P&percent_type=',
 'https://movie.douban.com/subject/1292052/comments?start=160&limit=20&sort=new_score&status=P&percent_type=',
 'https

In [16]:
len(cmt_urls)

250

In [18]:
all_comments = ''
for links in cmt_urls:
    for link in links:
        content = session.get(link, headers=headers, proxies={"http":"http://163.177.151.162:80"}) 
        all_comments += content.text
        time.sleep(1.5)
    time.sleep(2)

In [19]:
#利用lxml进行评论数据提取
selector = etree.HTML(all_comments)
comment_list = selector.xpath('//div[@class="comment-item"]')

In [20]:
#创建lists来保存评论信息
user_l = []
rating_l = []
user_comment_l = []
comment_time_l = []

In [22]:
#loop评论 保存每个评论的信息
for comment in comment_list:
    user = comment.xpath('.//span[@class="comment-info"]/a/text()')[0]
    rating = comment.xpath('.//span[@class="comment-info"]/span[2]/@class')[0][7:8]
    user_coomment = comment.xpath('.//div[@class="comment"]/p/text()')[0].strip()
    comment_time = comment.xpath('.//div[@class="comment"]/h3//span[@class="comment-time "]/text()')[0].strip()

    user_l.append(user)
    rating_l.append(rating)
    user_comment_l.append(user_coomment)
    comment_time_l.append(comment_time)

In [23]:
cmt_df = pd.DataFrame(list(zip(user_l, rating_l, user_comment_l, comment_time_l)))
cmt_df.head()

,0,1,2,3
0,kingfish,5,不需要女主角的好电影,2006-03-22
1,犀牛,5,当年的奥斯卡颁奖礼上，被如日中天的《阿甘正传》掩盖了它的光彩，而随着时间的推移，这部电影在越...,2005-10-28
2,711|湯好运,5,策划了19年的私奔……,2010-03-27
3,Eve|Classified,5,“这是一部男人必看的电影。”人人都这么说。但单纯从性别区分，就会让这电影变狭隘。《肖申克的救...,2008-05-09
4,艾小柯,5,关于希望最强有力的注释。,2006-06-20


In [24]:
cmt_df.shape

(17100, 4)

In [25]:
cmt_df.to_csv('./data/cmt_data.csv')

In [70]:
df = pd.read_csv('./data/cmt_data.csv',lineterminator='\n')

In [71]:
df.head()

,Unnamed: 0,0,1,2,3
0,0,kingfish,5,不需要女主角的好电影,2006-03-22
1,1,犀牛,5,当年的奥斯卡颁奖礼上，被如日中天的《阿甘正传》掩盖了它的光彩，而随着时间的推移，这部电影在越...,2005-10-28
2,2,711|湯好运,5,策划了19年的私奔……,2010-03-27
3,3,Eve|Classified,5,“这是一部男人必看的电影。”人人都这么说。但单纯从性别区分，就会让这电影变狭隘。《肖申克的救...,2008-05-09
4,4,艾小柯,5,关于希望最强有力的注释。,2006-06-20


In [72]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [74]:
df.columns = ['user', 'star', 'comment', 'time']

In [75]:
df.head()

,user,star,comment,time
0,kingfish,5,不需要女主角的好电影,2006-03-22
1,犀牛,5,当年的奥斯卡颁奖礼上，被如日中天的《阿甘正传》掩盖了它的光彩，而随着时间的推移，这部电影在越...,2005-10-28
2,711|湯好运,5,策划了19年的私奔……,2010-03-27
3,Eve|Classified,5,“这是一部男人必看的电影。”人人都这么说。但单纯从性别区分，就会让这电影变狭隘。《肖申克的救...,2008-05-09
4,艾小柯,5,关于希望最强有力的注释。,2006-06-20


In [76]:
df.to_csv('./data/cmt_data1.csv', index=False)

In [77]:
pd.read_csv('cmt_data1.csv', lineterminator='\n')

,user,star,comment,time
0,kingfish,5,不需要女主角的好电影,2006-03-22
1,犀牛,5,当年的奥斯卡颁奖礼上，被如日中天的《阿甘正传》掩盖了它的光彩，而随着时间的推移，这部电影在越...,2005-10-28
2,711|湯好运,5,策划了19年的私奔……,2010-03-27
3,Eve|Classified,5,“这是一部男人必看的电影。”人人都这么说。但单纯从性别区分，就会让这电影变狭隘。《肖申克的救...,2008-05-09
4,艾小柯,5,关于希望最强有力的注释。,2006-06-20
5,[已注销],5,有种鸟是关不住的.,2007-06-06
6,seawaver,5,"忒经典的东西,我要带去我的坟墓",2007-08-29
7,寂地,5,"超级喜欢超级喜欢,不看的话人生不圆满.",2006-01-02
8,独行狼,5,"这无疑是我看得最多的一部经典,爱死",2006-02-11
9,九尾黑猫,2,真的不喜欢，不好看，没感觉,2008-03-19
